# Example 2: How does global warming will affect urban climates?


In [1]:
from pyclmuapp import pts_clmu
from pyclmuapp import usp_clmu
from pyclmuapp.getcity import *
import matplotlib.pyplot as plt
import pandas as pd
import os

## 1 PTS mode: using the CESM provide global data


In [2]:
# the mask if from UrbanClimateExplorer/docs/notebooks/CESM1_get_urban_areas_mask.ipynb
# https://github.com/zhonghua-zheng/UrbanClimateExplorer/blob/main/docs/notebooks/CESM1_get_urban_areas_mask.ipynb
mask = pd.read_csv("src/CESM1_urban_mask.csv", index_col=0)
citylist = get_mask_cities(mask)


# set the location of London
London = {'lat': 51.5, 'lon': -0.1}

# find the closest city grid point to London (resolution is 1 degree, CESM nominal resolution)
closecity = closest(citylist,London)

closecity

{'lat': 51.361255645752, 'lon': 0.0}

In [3]:
# initialize the pts_clmu object

pts = pts_clmu(
    pwd=None,
    input_path="/home/zhonghua/p/scratch/CESMDATAROOT/inputdata",
    output_path="/home/zhonghua/p/scratch/CESMDATAROOT/Archive/lnd/hist",
    log_path="/home/zhonghua/p/scratch/CESMDATAROOT/CaseOutputs",
    scripts_path = "/home/zhonghua/p/scripts",
    container_type='docker')

# set the case configuration
pts.caseconfig['case_name'] = 'London'
pts.caseconfig['case_length'] = "3"
pts.caseconfig['hist_nhtfrq'] = '-24'
pts.caseconfig['STOP_OPTION'] = "nmonths"
pts.caseconfig['hist_mfilt']='240000'
pts.caseconfig['RUN_STARTDATE'] = '2012-06-01'
pts.caseconfig['DATM_CLMNCEP_YR_START'] = '2012'
pts.caseconfig['DATM_CLMNCEP_YR_END'] = '2012'
pts.caseconfig['case_lat'] = str(closecity['lat'])
pts.caseconfig['case_lon'] = str(closecity['lon'])

# run the case
pts_or=pts.run(iflog=True, cmdlogfile='./pts_London.log')

In [4]:
forcing = pts.input_path+"/atm/datm7/atm_forcing.datm7.cruncep_qianFill.0.5d.v7.c160715" + \
         "/TPHWL6Hrly" + "/clmforc.cruncep.V7.c2016.0.5d.TPQWL.2012-{month}.nc"

# modify the forcing
# Here, we modify the forcing to increase the temperature by 1 degree of forcing
# to simulate the global warming effect
# other forcing can be modified in the same way, e.g., precipitation, wind speed, etc.
# the action is a dict, the key is the forcing variable, the value is the modification value
# the modification value is the value to be added to the original forcing value
# the modification value can be negative, which means to decrease the original forcing value
action = {"TBOT":1.0}
action_r = {"TBOT":-1.0}

def modify_forcing(forcing, action):
    for month in range(6, 9):
        forcing_file = forcing.format(month=str(month).zfill(2))
        pts.modify_forcing(var="None", action=action, forcing_location=forcing_file)

modify_forcing(forcing, action)

# get the result
pts_warming = pts.run(iflog=True, cmdlogfile='./pts_London.log')

# recover the forcing
modify_forcing(forcing, action_r)

Print dataset location

In [5]:
print(pts_or)
print(pts_warming)

['/home/zhonghua/p/scratch/CESMDATAROOT/Archive/lnd/hist/London_hist0_2024-06-17_16-01-25_clm.nc']
['/home/zhonghua/p/scratch/CESMDATAROOT/Archive/lnd/hist/London_hist0_2024-06-17_16-01-56_clm.nc']


In [6]:
pts.case_clean()